In [ ]:
!pip install python-bidi
!pip install arabic_reshaper
!pip -q install hazm
!pip -q install clean-text[gpl]
!pip install bs4

     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 1.4 MB 48.2 MB/s 
     |████████████████████████████████| 233 kB 52.5 MB/s 
     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 170 kB 10.4 MB/s 
     |████████████████████████████████| 235 kB 44.0 MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving StopWords.txt to StopWords.txt
Saving upcoming comments.xlsx to upcoming comments.xlsx


--2021-11-07 06:09:47--  https://www.dropbox.com/s/fzvjblzeu0ztjq6/labeled%20comments%20up%20to%2020211017%201109.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fzvjblzeu0ztjq6/labeled%20comments%20up%20to%2020211017%201109.csv [following]
--2021-11-07 06:09:47--  https://www.dropbox.com/s/raw/fzvjblzeu0ztjq6/labeled%20comments%20up%20to%2020211017%201109.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf1882077a580e93de5e2e94553.dl.dropboxusercontent.com/cd/0/inline/BZi2u66fbCTPU7FaO4B8d5XhgbFGVuR9RrXIwTE8ovbDWYLutJsbzfczCLHqVb2aLOKPhMNxkgbwpuU0WsYBx0-z30SKjeu-t5a1wwtLYkCR80BksgP56snbBVIV4Fm1pZ3PuBKOnSjOfAUbfeP_1tDt/file# [following]
--2021-11-07 06:09:47--  https://ucf1882077a580e93de5e2e94553.dl.dropboxuser

# Data Preprocessing

In [ ]:
from pandas import *
from pandas import ExcelFile
import pandas as pd

#import ExcelFile
df = pd.read_csv('labeled comments up to 20211017 1109.csv')
# df = x.parse(x.sheet_names[0])

stop_words = open('StopWords.txt').read()
StopWords = stop_words.split("\n")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,14,15,16,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import arabic_reshaper
from hazm import Normalizer
from hazm import WordTokenizer
from cleantext import clean
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = WordTokenizer(join_verb_parts=False,replace_hashtags=False, replace_IDs=False)
normalizer = Normalizer(remove_extra_spaces=True, persian_numbers=False, persian_style=True,
                        punctuation_spacing=False, remove_diacritics=True,
                        affix_spacing=False, token_based=True)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
from hazm import Stemmer, Lemmatizer
from nltk import word_tokenize, sent_tokenize
# from PersianStemmer import PersianStemmer
from nltk.stem import WordNetLemmatizer
import string

from cleantext import clean
import codecs,re
def arabic_to_farsi(text):
    text = re.sub(r'[كﮑﮐﮏﮎﻜﻛﻚﻙ]', r'ک', text)
    text = re.sub(r'[ىىىﻴﻢﻳﻲﻱﻰىىﻯي]', r'ی', text)
    return text

def cleaning(text):
  text = str(text)
  text = re.sub(r'#', ' ', text)
  text = re.sub(r'_', ' ', text)
  text = text.replace('\u200c', ' ')
  text = re.sub(r'([ا-ی])\1{2,}', r'\1', text) # bbb+ -> b at least 2 char
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub(r'\@\w*', ' ', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(r'$', ' ', text)
  text = re.sub(r"[a-zA-Z0-9]+", " ",text)
  text = re.sub(r'<(.*?)>', ' ', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text)
  text=re.sub(r'[۰۱۲۳۴۵۶۷۸۹٠١١٢٣٤٥٦٧٧٨٨٩٩●]',' ',text)
  text=re.sub(r'[\·\♦\٭\\,\^\|\˝\٬\’\”\‹\▪\○¼ç½éêüəıœ™Ááàäāćíłñū©ٰٔ]',' ',text)
  text=re.sub(r'[ًٌٍَُِّْ]',' ',text)
  text=re.sub(r'[]',r' ',text)
  text=re.sub(r'[\–\—…°≈≠±≤≥\−×÷√٪→←↔↑↓\#\٫]',u' ',text) 
  text=text.replace(u'•',u' ').replace(u'ˈ',u' ').replace(u'؛',u' ').replace(u'/',u' ').replace(u'ۀ',u'هٔ').replace(u"﴿",u' ').replace(u"﴾",u' ').replace(u"'",u' ').replace(u'\\',u' ').replace(u'[',u' ').replace(u']',u' ').replace(u'?',u' ').replace(u'؟',u' ').replace(u')',u' ').replace(u'_',u' ').replace(u'(u',u' ').replace(u'}',u' ').replace(u'{',u' ').replace(u'.',u' ').replace(u'>',u' ').replace(u'<',u' ')
  text=text.replace(u'`',u' ').replace(u'\t',u' ').replace(u'=',u' ').replace(u'»',u' ').replace(u'«',u' ').replace(u'~',u' ').replace(u'!',u' ').replace(u'@',u' ').replace(u'$',u' ').replace(u',u',u' ').replace(u'%',u' ').replace(u'،',u' ').replace(u'-',u' ').replace(u';',u' ').replace(u':',u' ').replace(u'*',u' ').replace(u'"',u' ').replace(u'&',u' ').replace(u'#',u' ').replace(u'+',u' ')
  text=re.sub(r'[\n\r]{2,}',u'\n',text)
  text = re.sub(u'(\u202A|\u202B|\u202C|\u202D|\u202E|\u200F|\uFEFF|\u2003|\¬|\­)',u'\u200C', text)#حذف کارکترهای تغییرجهت
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'‌(?![ئاآأإژزرذدوؤةبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|ֹ)', u'', text) # در پس
  text = re.sub(u'(?<![ئبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|f|ֹ)‌', u'', text) # در پیش
  text=text.replace(u'­',u' ').replace(u'­',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'',u' ')
  text=text.replace(u'',u' ')
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'(\u00A0)',u' ', text).replace(u'(',u' ').replace(u')',u' ')
  text=text.replace(u'    ',u' ').replace(u'    ',u' ').replace(u'   ',u' ').replace(u'  ',u' ').replace(u'  ',u' ').replace(u'  ',u' ')
  text=arabic_to_farsi(text)
  return text.strip()

def remove_stopwords(text):
  tokens = word_tokenize(text)
  persian_stopwords = StopWords

  results = [w for w in tokens if w not in persian_stopwords and w not in string.punctuation]
  
  return ' '.join(results)

def stemming(text):
  tokens = word_tokenize(text)
  persian_stemmer = Stemmer()

  results = [persian_stemmer.stem(w) for w in tokens]

  return ' '.join(results)

def lemmatizing(text):
  wordlist = []
  lemmatizer = WordNetLemmatizer()
  
  sentences = sent_tokenize(text)

  for sentence in sentences:
        words=word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))

  return ' '.join(wordlist)

def normalize(text):
  result = lemmatizing(text)
  result = stemming(text)
  result = remove_stopwords(result)

  return result


In [ ]:
for i in range(len(df)):
  df.Comment[i] = str(df.Comment[i]).replace(df.BrandName[i],'')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df['Clean_Body'] = df.Comment.apply(cleaning)
df['normalized'] = df.Clean_Body.apply(normalize)

In [ ]:
df

,Id,BrandId,BrandName,Username,Date,Comment,CommentRaw,Rate,Social,SocialUrl,Status,Subject,ViewCount,LikeCount,PositiveAspects,NegativeAspects,NeutralAspects,TotalSentiment,Reason,UniqueId,LastUpdateDate,CheckingUserId,IsCurrent,IsValid,SecondCheckDate,Source,SourceId,Clean_Body,normalized
0,11,333.0,دیجی کالا,P_sherli_,2021-04-13 23:43:16.0000000,کل هفته منتظر بودم مامور اینارو بیاره :') الا...,کل هفته منتظر بودم مامور دیجی کالا اینارو بیار...,NaN,توئیتر,https://twitter.com/P_sherli_/statuses/1382049...,2,عمومی,0,0,NaN,NaN,NaN,0,تجربه نیست,fa7c8dae-c470-421f-9f2e-16bbced40324,2021-09-26 23:52:13.3636743,1161,NaN,NaN,NaN,NaN,NaN,کل هفته منتظر بودم مامور اینارو بیاره الانم نم...,هفته منتظر مامور اینارو بیاره نمیدون باهاشون چ...
1,12,333.0,دیجی کالا,Ali_rezanazari,2021-04-11 11:30:59.0000000,جالبه ترم جدید کلاس زبان 40 درصد نسبت به ترم ق...,جالبه ترم جدید کلاس زبان 40 درصد نسبت به ترم ق...,NaN,توئیتر,https://twitter.com/Ali_rezanazari/statuses/13...,2,عمومی,0,35,NaN,NaN,NaN,0,نام برند دیگر,0a084fee-a1dc-4c26-8da2-cc2750b0ad6a,2021-09-26 23:52:02.0166480,1161,NaN,NaN,NaN,NaN,NaN,جالبه ترم جدید کلاس زبان درصد نسبت به ترم قبل ...,جالبه تر کلاس زب درصد نسب تر افزا داشته درصد ت...
2,13,333.0,دیجی کالا,tousheh,2021-04-11 11:38:32.0000000,۲۲ فروردین: دو جوان رعنا از دیجیکالا آمدند و ل...,۲۲ فروردین: دو جوان رعنا از دیجیکالا آمدند و ل...,NaN,توئیتر,https://twitter.com/tousheh/statuses/138114210...,2,عمومی,0,129,NaN,NaN,NaN,0,تجربه نیست,019a733d-c684-4e79-bce2-f8a7939cbff2,2021-09-26 23:51:53.3006222,1161,NaN,NaN,NaN,NaN,NaN,فروردین دو جوان رعنا از دیجیکالا آمدند و لباسش...,فروردین جو رعنا دیجیکالا آمدند لباسشو مشکل خود...
3,14,333.0,دیجی کالا,MirorJo,2021-04-11 11:42:04.0000000,سوپر مارکت سر کوچه ما 😂\n هم طعم توت فرنگی و م...,سوپر مارکت سر کوچه ما 😂\nدیجی کالا هم طعم توت ...,NaN,توئیتر,https://twitter.com/MirorJo/statuses/138114299...,2,عمومی,0,0,NaN,NaN,NaN,0,تجربه نیست,152aba0c-b738-4ad3-8751-54f90e37eccc,2021-09-26 23:51:46.3961875,1161,NaN,NaN,NaN,NaN,NaN,سوپر مارکت سر کوچه ما \n هم طعم توت فرنگی و مو...,سوپر مارک کوچه طع فرنگ موز ریخته
4,15,333.0,دیجی کالا,Farenhid,2021-04-11 11:44:19.0000000,واقعا تو روزگاری که دعوا سر اینه که کالا یک سا...,واقعا تو روزگاری که دعوا سر اینه که کالا یک سا...,NaN,توئیتر,https://twitter.com/Farenhid/statuses/13811435...,2,عمومی,0,9,NaN,NaN,NaN,0,تجربه نیست,8139c0ac-8556-4998-a51f-a27c042ba891,2021-09-26 23:51:38.6683827,1161,NaN,NaN,NaN,NaN,NaN,واقعا تو روزگاری که دعوا سر اینه که کالا یک سا...,واقعا روزگار دعوا اینه کالا ساعته برسه ساعته ن...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34577,47569,3533.0,elia.shop1,s_abdolrahimi92,2021-09-07 07:36:26.0000000,چرا دایرکت جواب نمیدید؟از دیشب من نتونستم خرید...,چرا دایرکت جواب نمیدید؟از دیشب من نتونستم خرید...,NaN,اینستاگرام,NaN,3,NaN,0,0,NaN,NaN,NaN,0,NaN,5c67f51a-c9f8-4998-b0d0-1e4546116ea6,2021-10-17 11:00:32.3265163,401,0.0,1.0,NaN,https://www.instagram.com/p/CTfdwyogV1x,637665969860000000_s_abdolrahimi92,چرا دایرکت جواب نمیدید از دیشب من نتونستم خرید...,دایرک جواب نمیدید دیشب نتونس خرید ازتون
34578,47609,3485.0,padmira.ir,zahrash7594,2021-09-08 14:44:58.0000000,خیلی خوشگله من امروز سفارشم رسید وسوسه شدم دوب...,خیلی خوشگله من امروز سفارشم رسید وسوسه شدم دوب...,4.0,اینستاگرام,NaN,1,NaN,0,0,NaN,NaN,NaN,1,NaN,077fcc40-d093-4217-a4de-a6648fd6bae4,2021-10-16 16:57:52.8704909,401,0.0,1.0,NaN,https://www.instagram.com/p/CTkG8ELoHrA,637667090980000000_zahrash7594,خیلی خوشگله من امروز سفارشم رسید وسوسه شدم دوب...,خیل خوشگله رسید وسوسه دوباره بخر
34579,47610,3485.0,padmira.ir,aylar._.at,2021-09-08 14:37:49.0000000,هزینه پست پیشتاز رو میگیرید ولی خیلی طول میکشه...,هزینه پست پیشتاز رو میگیرید ولی خیلی طول میکشه...,2.0,اینستاگرام,NaN,1,NaN,0,0,NaN,NaN,NaN,-1,NaN,64525b9c-6288-4390-869b-81832bbd6e6a,2021-10-16 15:55:12.6255675,401,0.0,1.0,NaN,https://www.instagram.com/p/CTkG8ELoHrA,637667086690000000_aylar._.at,هزینه پست پیشتاز رو میگیرید ولی خیلی طول میکشه...,هزینه پیشتاز میگیرید خیل طول

# Document Term Matrix

In [ ]:
import pandas as pd

In [ ]:
sentences = df['normalized']

In [ ]:
all_words = []
for sentence in sentences:
  all_words += word_tokenize(sentence)
all_words = list(dict.fromkeys(all_words))

In [ ]:
all_valid_words = [x for x in all_words if len(x) != 1]

In [ ]:
matrix = [[0 for i in range(len(all_words))] for j in range(len(sentences))]

In [ ]:
for i in range(len(sentences)):
  tokenized_sent = word_tokenize(sentences[i])
  for j in range(len(all_words)):
    if all_words[j] in tokenized_sent:
      matrix[i][j] = 1

In [ ]:
data = pd.DataFrame(matrix, columns=all_words)

In [ ]:
data

,بودجه,ندار,ارگان,سازمان,حما,کنه,حساب,تون,سرنگ,خال,داروخانه,سرکوچه,فله,بخر,آب,لیمو,خر,هزینه,تکیه,طب,سنت,پنج,تن,آل,عبا,نتیجه,خور,رس,میلیارد,نیاز,فاز,اول,اوک,شه,گردو,پوس,خرید,کیلو,تومن,مژده,...,غیراصل,بدند,جهیزیه,بااینکه,برجسته,هاره,نظرتو,بردم,زیبو,بفهمونه,هواار,تخفیفشون,میرسونید,حرفتون,امده,تاسف,آضافه,ویژن,دونند,امنیتتون,معلول,فراد,فرشا,بدقلق,بپسند,مسترد,نکردین,پشته,آیند,مسلمان,نشست,دین,ربا,کمروشو,مسئولیته,نذاشتن,کاذب,واردکننده,هنچز,منچس
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8481,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8482,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# LDA

In [ ]:
doc_clean = [word_tokenize(x) for x in df['normalized']]

In [ ]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.015*"خرید" + 0.013*"کالا" + 0.012*"کنید"'), (1, '0.014*"ساع" + 0.008*"شهر" + 0.008*"دس"'), (2, '0.011*"خونه" + 0.010*"خیل" + 0.009*"خرید"')]


# LSA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
documents = [x for x in df['normalized']]  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

In [ ]:
svd_matrix

array([[ 0.00177024,  0.05597934, -0.02521203, ...,  0.00119176,
        -0.03011514, -0.01952095],
       [ 0.00255807,  0.13021651, -0.00682225, ...,  0.00047145,
        -0.00433624, -0.00684824],
       [ 0.00185091,  0.09119677, -0.01515999, ...,  0.00646919,
         0.02542615,  0.00176782],
       ...,
       [ 0.00333121,  0.18503061,  0.08491878, ..., -0.04176867,
         0.04011342,  0.05014544],
       [ 0.00236996,  0.10602848, -0.04924988, ...,  0.06207778,
         0.00063644, -0.06555047],
       [ 0.00273118,  0.09394266, -0.05138182, ...,  0.04392278,
         0.0108855 , -0.0776929 ]])

# Word and Sentence Embeddings

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sentences = [x for x in df['normalized']]

In [ ]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

## Doc2vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def sentence_embedding(tokenized_sent):
  tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]
  model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)
  model.wv.vocab

  return model

In [ ]:
# test
# test_doc = word_tokenize("سلام من حالم خوب است".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])

## Word2vec

In [ ]:
from gensim.models import Word2Vec
def word_embedding(sentences):
  a = [word_tokenize(x) for x in sentences]
  model = Word2Vec(sentences=a, window=5, min_count=1, workers=4)

  return model

In [ ]:
def compare_embeddings(word, tokenized_sent, sentences):
  word_model = word_embedding(sentences)
  sentence_model = sentence_embedding(tokenized_sent)

  print(f"word2vec:\n {word}:{word_model.wv.most_similar(word, topn=5)}")
  print(f"doc2vec:\n {word}:{sentence_model.wv.most_similar(word, topn=5)}")

In [ ]:
compare_embeddings("سلام", tokenized_sent, sentences)

word2vec:
 سلام:[('پا', 0.9996176362037659), ('متاسفانه', 0.9995843768119812), ('خودشون', 0.9995696544647217), ('خو', 0.9995650053024292), ('شون', 0.9995605945587158)]
doc2vec:
 سلام:[('محتر', 0.8016477823257446), ('متفرقه', 0.7814697623252869), ('اوه', 0.7577250003814697), ('بازرگان', 0.7502493262290955), ('زیس', 0.7305694818496704)]
